# Predict if a YouTube comment is a spam
### Import the data as pandas DataFrame

In [1]:
import os
from keras.utils.data_utils import get_file
import cPickle as pickle
import bcolz


def get_glove_dataset(dataset):
    md5sums = {'6B.50d': '8e1557d1228decbda7db6dfd81cd9909',
               '6B.100d': 'c92dbbeacde2b0384a43014885a60b2c',
               '6B.200d': 'af271b46c04b0b2e41a84d8cd806178d',
               '6B.300d': '30290210376887dcc6d0a5a6374d8255'}
    glove_path = os.path.abspath('data/glove/results')
    %mkdir -p $glove_path
    return get_file(dataset,
                    'http://files.fast.ai/models/glove/' + dataset + '.tgz',
                    cache_subdir=glove_path,
                    md5_hash=md5sums.get(dataset, None),
                    untar=True)

def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb')),
        pickle.load(open(loc+'_idx.pkl','rb')))

def load_array(fname):
    return bcolz.open(fname)[:]

vecs, words, wordidx = load_vectors(get_glove_dataset('6B.50d'))

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


In [2]:
from collections import Counter
import itertools
import nltk
import re
import pandas as pd
import numpy as np

train_filenames = ['Youtube01-Psy.csv', 'Youtube02-KatyPerry.csv', 'Youtube03-LMFAO.csv', 'Youtube04-Eminem.csv']
valid_filename = 'Youtube05-Shakira.csv'

train_df = pd.concat([pd.read_csv('data/' + filename, encoding='utf-8-sig') for filename in train_filenames])

train_df.CONTENT.head()

def replace_url(phrase):
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', phrase)
    for url in urls:
        phrase = phrase.replace(url, 'LINKHTTP')
    return phrase

def format_phrase(phrase):
    phrase = replace_url(phrase)
    words = re.sub("[^\w]", " ",  phrase).split()
    words = nltk.word_tokenize(phrase)
    return [w.replace(" ", "").lower() for w in words]
    
def get_unique_words(phrases):
    words_list = phrases.sum()
    return np.unique(np.array(words_list))

def words2idxs(phrase):
    words_count = len(wordidx) - 1
    return [wordidx[word] if word in wordidx else words_count for word in phrase]

train_df = train_df.assign(CONTENT_WORDS=train_df.CONTENT.apply(format_phrase))

#unique_words = get_unique_words(train_df.CONTENT_WORDS)
#word2idx = {v: k for k, v in enumerate(unique_words)}

train_df.head(3)

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS,CONTENT_WORDS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1,"[huh, ,, anyway, check, out, this, you, [, tub..."
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1,"[hey, guys, check, out, my, new, channel, and,..."
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1,"[just, for, test, i, have, to, say, murdev.com]"


### Get words indexes

In [3]:
format_phrase('Agnes Blog is totALLy awesome :) !!!!')

['agnes', 'blog', 'is', 'totally', 'awesome', ':', ')', '!', '!', '!', '!']

In [4]:
from keras import backend as K
from keras.preprocessing import sequence

train_df = train_df.assign(CONTENT_IDX=train_df.CONTENT_WORDS.apply(words2idxs))

maxlen = train_df.CONTENT_IDX.map(len).max()
train_content_idx = sequence.pad_sequences(train_df.CONTENT_IDX, maxlen=maxlen, value=-1)

In [5]:
valid_df = pd.read_csv('data/' + valid_filename, encoding='utf-8-sig')
valid_df = valid_df.assign(CONTENT_WORDS=valid_df.CONTENT.apply(format_phrase))
valid_df = valid_df.assign(CONTENT_IDX=valid_df.CONTENT_WORDS.apply(words2idxs))
valid_content_idx = sequence.pad_sequences(valid_df.CONTENT_IDX, maxlen=maxlen, value=-1)

In [6]:
from keras.layers import MaxPooling1D, Conv1D, BatchNormalization
from keras.layers import Flatten, Dense, Embedding, Dropout, Dense, SpatialDropout1D
from keras.models import Sequential
from keras.optimizers import Adam, Adamax, RMSprop, SGD
from keras.regularizers import l2

#embeddings_regularizer=l2(1e-4),

vocab_size = len(wordidx)
vgg_model = Sequential([
    
    Embedding(vocab_size, 50, input_length=maxlen, embeddings_regularizer=l2(1e-4), 
              dropout=0.2, weights=[vecs], trainable=False),
    
    # Conv Block 1
    Conv1D(64, 5, padding='same', activation='relu'),
    #Conv1D(64, 3, padding='same', activation='relu'),
    MaxPooling1D(),
    Dropout(0.5),
    
    # Conv Block 2
    #Conv1D(128, 3, padding='same', activation='relu'),
    #Conv1D(128, 3, padding='same', activation='relu'),
    #MaxPooling1D(),
    #Dropout(0.6),
        
    # FC layers wiht BatchNorm
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(100, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')])


vgg_model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
vgg_model.optimizer.lr = 10e-5
vgg_model.fit(train_content_idx, train_df.CLASS, validation_data=(valid_content_idx, valid_df.CLASS), 
              epochs=3, batch_size=64)

vgg_model.optimizer.lr = 10e-3
vgg_model.fit(train_content_idx, train_df.CLASS, validation_data=(valid_content_idx, valid_df.CLASS), 
              epochs=10, batch_size=64)
vgg_model.optimizer.lr = 10e-4
vgg_model.fit(train_content_idx, train_df.CLASS, validation_data=(valid_content_idx, valid_df.CLASS), 
              epochs=40, batch_size=64)
vgg_model.optimizer.lr = 10e-5
vgg_model.fit(train_content_idx, train_df.CLASS, validation_data=(valid_content_idx, valid_df.CLASS), 
              epochs=40, batch_size=64)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:13: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.


Train on 1586 samples, validate on 370 samples
Epoch 1/3
1586/1586 [==============================] - 2s - loss: 831.5376 - acc: 0.5076 - val_loss: 831.3093 - val_acc: 0.4703
Epoch 2/3
1586/1586 [==============================] - 2s - loss: 831.4612 - acc: 0.5485 - val_loss: 831.2034 - val_acc: 0.7054
Epoch 3/3
1586/1586 [==============================] - 2s - loss: 831.4791 - acc: 0.5448 - val_loss: 831.2057 - val_acc: 0.7459
Train on 1586 samples, validate on 370 samples
Epoch 1/10
1586/1586 [==============================] - 2s - loss: 831.4795 - acc: 0.5347 - val_loss: 831.1635 - val_acc: 0.7243
Epoch 2/10
1586/1586 [==============================] - 2s - loss: 831.4486 - acc: 0.5492 - val_loss: 831.2000 - val_acc: 0.5270
Epoch 3/10
1586/1586 [==============================] - 2s - loss: 831.4771 - acc: 0.5378 - val_loss: 831.1841 - val_acc: 0.7514
Epoch 4/10
1586/1586 [==============================] - 2s - loss: 831.4886 - acc: 0.5359 - val_loss: 831.1649 - val_acc: 0.7162
Epoch 

In [7]:
non_spams = ['lol love it', 'awesome video', 'i love this song', 'so many views', 'she must have so much money']
spams = ['check my channel', 'want to have more money contact me mail', 'email me to earn a lot of money',
         'email me to at agne@gmal.com', 'suscribe to my yt channel', 'http://salut.com']

spams = [words2idxs(format_phrase(spam)) for spam in spams]
non_spams = [words2idxs(format_phrase(spam)) for spam in non_spams]


spams = sequence.pad_sequences(spams, maxlen=maxlen, value=-1)
non_spams = sequence.pad_sequences(non_spams, maxlen=maxlen, value=-1)

vgg_model.predict(non_spams)

array([[ 0.13810994],
       [ 0.37778828],
       [ 0.14262445],
       [ 0.06725907],
       [ 0.26699576]], dtype=float32)

In [8]:
vgg_model.predict(spams)

array([[ 0.99768353],
       [ 0.9818902 ],
       [ 0.96827245],
       [ 0.98142713],
       [ 0.97258818],
       [ 0.49787214]], dtype=float32)